# Read TSV file and write Arrow

Attach the packages to be used

In [1]:
using Arrow, CSV, DataFrames

Read the tab-separated-value file

In [2]:
df = CSV.read(
    "ncvoter_Statewide.txt",
    DataFrame;              # return as a DataFrame
    delim='\t',             # tab-separated values
    dateformat="mm/dd/yyyy",
    missingstring=["", " ", "##/##/####"],
    downcast=true,          # use small integer sizes if possible
    pool=0.3,               # use PooledArrays if less than 30% distinct
    stripwhitespace=true,
    stringtype=String,      # avoid complexities from special string types
)

Row,county_id,county_desc,voter_reg_num,ncid,last_name,first_name,middle_name,name_suffix_lbl,status_cd,voter_status_desc,reason_cd,voter_status_reason_desc,res_street_address,res_city_desc,state_cd,zip_code,mail_addr1,mail_addr2,mail_addr3,mail_addr4,mail_city,mail_state,mail_zipcode,full_phone_number,confidential_ind,registr_dt,race_code,ethnic_code,party_cd,gender_code,birth_year,age_at_year_end,birth_state,drivers_lic,precinct_abbrv,precinct_desc,municipality_abbrv,municipality_desc,ward_abbrv,ward_desc,cong_dist_abbrv,super_court_abbrv,judic_dist_abbrv,nc_senate_abbrv,nc_house_abbrv,county_commiss_abbrv,county_commiss_desc,township_abbrv,township_desc,school_dist_abbrv,school_dist_desc,fire_dist_abbrv,fire_dist_desc,water_dist_abbrv,water_dist_desc,sewer_dist_abbrv,sewer_dist_desc,sanit_dist_abbrv,sanit_dist_desc,rescue_dist_abbrv,rescue_dist_desc,munic_dist_abbrv,munic_dist_desc,dist_1_abbrv,dist_1_desc,vtd_abbrv,vtd_desc
,Int8,String,Int32,String,String,String?,String?,String?,String,String,String?,String?,String,String?,String?,Int32?,String?,String?,String?,String?,String?,String?,Int32?,String?,String,Date?,String?,String,String,String?,Int16,Int16,String?,String,String?,String?,String?,String?,String?,String?,Int8?,String?,String?,Int8?,Int8?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,Int8?,String?,String?,String?
1,1,ALAMANCE,9005990,AA56273,AABEL,RUTH,EVELYN,missing,R,REMOVED,RD,DECEASED,REMOVED,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,N,1984-10-01,W,NL,UNA,F,1935,90,NY,N,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
2,1,ALAMANCE,9178574,AA201627,AARDEN,JONI,AUTUMN,missing,R,REMOVED,RL,MOVED FROM COUNTY,REMOVED,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,N,2018-02-23,W,UN,UNA,F,1978,47,DC,Y,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
3,1,ALAMANCE,9205561,AA216996,AARMSTRONG,TIMOTHY,DUANE,missing,A,ACTIVE,A2,CONFIRMATION PENDING,3670 COVINGTON TRL,MEBANE,NC,27302,3670 COVINGTON TRL,missing,missing,missing,MEBANE,NC,27302,missing,N,2020-10-31,W,UN,REP,M,1966,59,AL,N,103,MELVILLE 3,missing,missing,missing,missing,9,17,17,25,64,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,17,PROSECUTORIAL DISTRICT 17,103,103
4,1,ALAMANCE,9048723,AA98377,AARON,CHRISTINA,CASTAGNA,missing,A,ACTIVE,AV,VERIFIED,421 WHITT AVE,BURLINGTON,NC,27215,421 WHITT AVE,missing,missing,missing,BURLINGTON,NC,27215,3362291110,N,1996-03-26,W,UN,UNA,F,1976,49,NC,Y,03SE,SOUTH BOONE E,BUR,BURLINGTON,missing,missing,9,17,17,25,63,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,BUR,BURLINGTON,17,PROSECUTORIAL DISTRICT 17,03S,03S
5,1,ALAMANCE,9019674,AA69747,AARON,CLAUDIA,HAYDEN,missing,A,ACTIVE,AV,VERIFIED,1013 EDITH ST,BURLINGTON,NC,27215,1013 EDITH ST,missing,missing,missing,BURLINGTON,NC,27215,2228834,N,1989-08-15,W,NL,UNA,F,1945,80,VA,Y,124,BURLINGTON 4,BUR,BURLINGTON,missing,missing,9,17,17,25,63,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,BUR,BURLINGTON,17,PROSECUTORIAL DISTRICT 17,124,124
6,1,ALAMANCE,9129589,AA170513,AARON,JAMES,MICHAEL,missing,A,ACTIVE,AV,VERIFIED,5608 OLD CHEROKEE LN,GRAHAM,NC,27253,PO BOX 98,missing,missing,missing,SAXAPAHAW,NC,27340,2027443411,N,2012-03-0

Summary of values in table

In [3]:
desc = describe(df, :min, :max, :nunique, :nmissing, :eltype)
desc[1:25, :]

Row,variable,min,max,nunique,nmissing,eltype
,Symbol,Any,Any,Union…,Int64,Type
1,county_id,1,100,,0,Int8
2,county_desc,ALAMANCE,YANCEY,100,0,String
3,voter_reg_num,0,1000931695,,0,Int32
4,ncid,AA100000,ES9999,8778585,0,String
5,last_name,CARROLL,ZYWOT,511478,0,String
6,first_name,GARLANDA,ZZREAM,336245,120,"Union{Missing, String}"
7,middle_name,BULLABOY,ZYZO,461538,742250,"Union{Missing, String}"
8,name_suffix_lbl,E,X,17,8399078,"Union{Missing, String}"
9,status_cd,A,S,5,0,String


In [4]:
desc[26:50,:]

Row,variable,min,max,nunique,nmissing,eltype
,Symbol,Any,Any,Union…,Int64,Type
1,registr_dt,1938-01-01,2025-01-24,23255,6004,"Union{Missing, Date}"
2,race_code,A,W,8,1,"Union{Missing, String}"
3,ethnic_code,HL,UN,3,0,String
4,party_cd,CST,WTP,9,0,String
5,gender_code,F,U,3,12,"Union{Missing, String}"
6,birth_year,1753,2008,,0,Int16
7,age_at_year_end,17,272,,0,Int16
8,birth_state,AK,WY,57,2120862,"Union{Missing, String}"
9,drivers_lic,N,Y,2,0,String


In [5]:
desc[51:end, :]

Row,variable,min,max,nunique,nmissing,eltype
,Symbol,Any,Any,Union…,Int64,Type
1,school_dist_desc,#01,WHITEVILLE SCHOOL DISTRICT 4,180,2668915,"Union{Missing, String}"
2,fire_dist_abbrv,01,ZION,128,8424323,"Union{Missing, String}"
3,fire_dist_desc,ADR FIRE DISTRICT,ZIONVILLE,146,8424323,"Union{Missing, String}"
4,water_dist_abbrv,01,WD00,22,8638653,"Union{Missing, String}"
5,water_dist_desc,CENTRAL,WOODFIN WATER DISTRICT,20,8638653,"Union{Missing, String}"
6,sewer_dist_abbrv,0001,LP,8,8754550,"Union{Missing, String}"
7,sewer_dist_desc,DISTRICT 1,TUNIS,8,8754550,"Union{Missing, String}"
8,sanit_dist_abbrv,1,WSD,31,8659822,"Union{Missing, String}"
9,sanit_dist_desc,1ST SANITARY,WHITTIER SANITARY DISTRICT,31,8659822,"Union{Missing, String}"


There are peculiarities in this table (birth_year of 1753, coding glitches) but we will ignore them for the time being.

In [6]:
afn = Arrow.write("ncvoter_Statewide.arrow", df);   # uncompressed Arrow file

Next, read it back in to Julia.  Because we did not use compression of the columns the file can be memory-mapped to be read, which is very fast.

In [7]:
tbl = Arrow.Table(afn)

Arrow.Table with 8778585 rows, 67 columns, and schema:
 :county_id          Int8
 :county_desc        String
 :voter_reg_num      Int32
 :ncid               String
 :last_name          String
 :first_name         Union{Missing, String}
 :middle_name        Union{Missing, String}
 :name_suffix_lbl    Union{Missing, String}
 :status_cd          String
 :voter_status_desc  String
 ⋮                   
 :sanit_dist_desc    Union{Missing, String}
 :rescue_dist_abbrv  Union{Missing, String}
 :rescue_dist_desc   Union{Missing, String}
 :munic_dist_abbrv   Union{Missing, String}
 :munic_dist_desc    Union{Missing, String}
 :dist_1_abbrv       Union{Missing, Int8}
 :dist_1_desc        Union{Missing, String}
 :vtd_abbrv          Union{Missing, String}
 :vtd_desc           Union{Missing, String}